# OpenEvolve Standalone Tutorial

This notebook demonstrates how to use **GEAK-OpenEvolve** for GPU kernel optimization using LLM-guided evolution.

## Prerequisites
 **Environment Variables**: Set `OPENAI_API_KEY`

## What You'll Learn

- How to set up GEAK-OpenEvolve
- How to prepare an initial kernel program
- How to configure evolution parameters
- How to run the evolution pipeline
- How to analyze results

In [1]:
# Step 1: Environment Setup
import os
import sys
from pathlib import Path

# Get geak-openevolve root
OPENEVOLVE_ROOT = Path.cwd().parent
print(f"OpenEvolve Root: {OPENEVOLVE_ROOT}")

# Add to Python path
if str(OPENEVOLVE_ROOT) not in sys.path:
    sys.path.insert(0, str(OPENEVOLVE_ROOT))

print(f"\n✅ OpenEvolve root: {OPENEVOLVE_ROOT}")
print(f"✅ Python path updated")


OpenEvolve Root: /home/sapmajum/neurips/geak-openevolve

✅ OpenEvolve root: /home/sapmajum/neurips/geak-openevolve
✅ Python path updated


In [2]:
# Step 1.5: Clone and Install GEAK-eval (if not already done)
import os
from pathlib import Path
import subprocess

OPENEVOLVE_ROOT = Path.cwd().parent
GEAK_EVAL_DIR = OPENEVOLVE_ROOT / "GEAK-eval-OE"

if not GEAK_EVAL_DIR.exists():
    print("📥 Cloning GEAK-eval...")
    os.chdir(OPENEVOLVE_ROOT)
    
    # Clone and checkout
    subprocess.run(["git", "clone", "git@github.com:AMD-AGI/GEAK-eval.git", "GEAK-eval-OE"], check=True)
    os.chdir("GEAK-eval-OE")
    subprocess.run(["git", "checkout", "geak-oe"], check=True)
    
    print("✅ GEAK-eval cloned")
    
    # Install
    print("📦 Installing GEAK-eval...")
    subprocess.run(["pip", "install", "-e", ".", "--no-deps"], check=True)
    print("✅ GEAK-eval installed")
else:
    print(f"✅ GEAK-eval already exists at: {GEAK_EVAL_DIR}")
    
    # Check if installed
    try:
        result = subprocess.run(["which", "geak-eval"], capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ geak-eval command available: {result.stdout.strip()}")
        else:
            print("⚠️  geak-eval command not found, installing...")
            os.chdir(GEAK_EVAL_DIR)
            subprocess.run(["pip", "install", "-e", ".", "--no-deps"], check=True)
            print("✅ GEAK-eval installed")
    except Exception as e:
        print(f"⚠️  Could not check geak-eval: {e}")


✅ GEAK-eval already exists at: /home/sapmajum/neurips/geak-openevolve/GEAK-eval-OE
✅ geak-eval command available: /home/sapmajum/miniconda3/bin/geak-eval


In [10]:
# Step 2: Set Environment Variables
import os
from pathlib import Path

# Set API key
# os.environ['OPENAI_API_KEY'] = "<your-api-key>"

# Set ROCM_GOLDEN_DATA_PATH
OPENEVOLVE_ROOT = Path.cwd().parent
GOLDEN_DATA_PATH = OPENEVOLVE_ROOT / "GEAK-eval-OE/geak_eval/data/ROCm/data/performance/golden_results"
os.environ['ROCM_GOLDEN_DATA_PATH'] = str(GOLDEN_DATA_PATH)

print(f"✅ OPENAI_API_KEY set")
print(f"✅ ROCM_GOLDEN_DATA_PATH = {GOLDEN_DATA_PATH}")
print(f"   Path exists: {GOLDEN_DATA_PATH.exists()}")


✅ OPENAI_API_KEY set
✅ ROCM_GOLDEN_DATA_PATH = /home/sapmajum/neurips/geak-openevolve/GEAK-eval-OE/geak_eval/data/ROCm/data/performance/golden_results
   Path exists: True


In [11]:
# Step 3: Verify OpenEvolve Installation
import sys
import torch

print(f"Python: {sys.version.split()[0]}")
print(f"PyTorch: {torch.__version__}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}")

try:
    import triton
    print(f"Triton: {triton.__version__}")
except:
    print("❌ Triton not found")

try:
    import openevolve
    print(f"OpenEvolve: {openevolve.__version__ if hasattr(openevolve, '__version__') else 'installed'}")
except:
    print("❌ OpenEvolve not found - install with: pip install -e .")

print("\n✅ Environment ready!")


Python: 3.13.9
PyTorch: 2.7.0.dev20250310+rocm6.2.4
GPU: AMD Instinct MI325X
Triton: 3.2.0
OpenEvolve: 0.1.0

✅ Environment ready!


## Kernel Preparation

OpenEvolve requires:
1. **Initial Kernel**: The starting kernel code to optimize
2. **Evaluator**: A function that evaluates kernel performance
3. **Configuration**: Evolution parameters (iterations, population size, etc.)

We'll use a validated ROCm Triton kernel as our example.


In [12]:
# Step 4: Select Example Kernel
from pathlib import Path

OPENEVOLVE_ROOT = Path.cwd().parent
TUTORIAL_DIR = OPENEVOLVE_ROOT / "tutorial"

# Use kernel from GEAK-eval-OE (cloned GEAK-eval repository)
INITIAL_KERNEL = OPENEVOLVE_ROOT / "GEAK-eval-OE/geak_eval/data/ROCm/data/ROCm_v1/test_add_kernel.py"

if INITIAL_KERNEL.exists():
    print(f"✅ Selected kernel: {INITIAL_KERNEL.name}")
    print(f"   Path: {INITIAL_KERNEL.relative_to(OPENEVOLVE_ROOT)}")
    
    # Quick peek at the kernel
    with open(INITIAL_KERNEL, 'r') as f:
        lines = f.readlines()
    
    # Find the kernel function
    in_kernel = False
    kernel_lines = []
    for line in lines:
        if '@triton.jit' in line:
            in_kernel = True
        if in_kernel:
            kernel_lines.append(line.rstrip())
            if line.strip().startswith('tl.store') and 'output' in line:
                break
    
    print(f"\n📝 Kernel Preview:")
    for line in kernel_lines[:15]:
        print(f"   {line}")
    if len(kernel_lines) > 15:
        print(f"   ... ({len(kernel_lines)-15} more lines)")
else:
    print(f"❌ Kernel not found at: {INITIAL_KERNEL}")
    INITIAL_KERNEL = None


✅ Selected kernel: test_add_kernel.py
   Path: GEAK-eval-OE/geak_eval/data/ROCm/data/ROCm_v1/test_add_kernel.py

📝 Kernel Preview:
   @triton.jit
   def add_kernel(
       x_ptr,
       y_ptr,
       output_ptr,
       n_elements,
       BLOCK_SIZE: tl.constexpr,
   ):
       pid = tl.program_id(axis=0)  # We use a 1D launch grid so axis is 0.
       block_start = pid * BLOCK_SIZE
       offsets = block_start + tl.arange(0, BLOCK_SIZE)
       mask = offsets < n_elements
   
       x_block_ptr = tl.make_block_ptr(base=x_ptr, shape=(n_elements, ), strides=(1, ), offsets=(pid * BLOCK_SIZE, ),
                                       block_shape=(BLOCK_SIZE, ), order=(0, ))
   ... (5 more lines)


In [13]:
# Step 5: Setup Evaluator
from pathlib import Path

OPENEVOLVE_ROOT = Path.cwd().parent

# Use the ROCm evaluator from examples
EVALUATOR_PATH = OPENEVOLVE_ROOT / "examples/tb/rocm_evaluator.py"

if EVALUATOR_PATH.exists():
    print(f"✅ Using evaluator: {EVALUATOR_PATH.name}")
    print(f"   Path: {EVALUATOR_PATH.relative_to(OPENEVOLVE_ROOT)}")
else:
    print(f"❌ Evaluator not found at: {EVALUATOR_PATH}")
    EVALUATOR_PATH = None


✅ Using evaluator: rocm_evaluator.py
   Path: examples/tb/rocm_evaluator.py


In [14]:
# Step 6: Configure Evolution Parameters
import yaml
from pathlib import Path

OPENEVOLVE_ROOT = Path.cwd().parent
TUTORIAL_DIR = OPENEVOLVE_ROOT / "tutorial"

# Configuration parameters - EASILY ADJUSTABLE
MAX_ITERATIONS = 3
POPULATION_SIZE = 50
NUM_ISLANDS = 4
LOG_LEVEL = "WARNING"

# Try multiple config templates
CONFIG_TEMPLATES = [
    OPENEVOLVE_ROOT / "configs/default_config.yaml",
    OPENEVOLVE_ROOT / "examples/tb/configs/demo_config.yaml",
]

CONFIG_FILE = TUTORIAL_DIR / "tutorial_config.yaml"

# Find first available template
template_found = None
for template in CONFIG_TEMPLATES:
    if template.exists():
        template_found = template
        print(f"✅ Found config template: {template.relative_to(OPENEVOLVE_ROOT)}")
        break

if template_found:
    with open(template_found, 'r') as f:
        config = yaml.safe_load(f)
    
    config['max_iterations'] = MAX_ITERATIONS
    config['log_level'] = LOG_LEVEL
    
    if 'database' not in config:
        config['database'] = {}
    config['database']['population_size'] = POPULATION_SIZE
    config['database']['num_islands'] = NUM_ISLANDS
    config['database']['log_prompts'] = True
    
    # CRITICAL: Fix db_path (can't be None)
    if config['database'].get('db_path') is None:
        config['database']['db_path'] = 'program_database'
    
    if 'llm' not in config:
        config['llm'] = {}
    
    # CRITICAL: Set sampling configuration
    config['llm']['sampling'] = {'fn': 'random'}
    
    config['llm']['models'] = [{'name': 'claude-sonnet-4', 'weight': 1.0}]
    config['llm']['evaluator_models'] = [{'name': 'claude-sonnet-4', 'weight': 1.0}]
    config['llm']['api_base'] = 'https://llm-api.amd.com/AnthropicVertex/deployments/claude-sonnet-4'
    config['llm']['api_key'] = None
    
    if 'evaluator' not in config:
        config['evaluator'] = {}
    config['evaluator']['cascade_evaluation'] = False
    config['evaluator']['verbose'] = False
    
    config['diff_based_evolution'] = True
    config['max_code_length'] = 50000
    
    # CRITICAL: Create evals directory for evaluator temp files
    evals_dir = TUTORIAL_DIR / "evals"
    evals_dir.mkdir(exist_ok=True)
    
    with open(CONFIG_FILE, 'w') as f:
        yaml.dump(config, f, default_flow_style=False, sort_keys=False)
    
    print(f"✅ Configuration saved to: {CONFIG_FILE.name}")
    print(f"\n📝 Evolution Parameters:")
    print(f"  Max Iterations:  {MAX_ITERATIONS}")
    print(f"  Population Size: {POPULATION_SIZE}")
    print(f"  Num Islands:     {NUM_ISLANDS}")
    print(f"  Log Level:       {LOG_LEVEL}")
    print(f"  LLM Model:       claude-sonnet-4")
    print(f"  LLM Sampling:    random")
    print(f"  Database Path:   {config['database']['db_path']}")
    print(f"\n✅ Ready to run evolution!")
else:
    print("❌ No config template found!")
    CONFIG_FILE = None


✅ Found config template: configs/default_config.yaml
✅ Configuration saved to: tutorial_config.yaml

📝 Evolution Parameters:
  Max Iterations:  3
  Population Size: 50
  Num Islands:     4
  Log Level:       WARNING
  LLM Model:       claude-sonnet-4
  LLM Sampling:    random
  Database Path:   program_database

✅ Ready to run evolution!


In [8]:
# Step 7: Setup Output Directory and Validate
from pathlib import Path
from datetime import datetime

OPENEVOLVE_ROOT = Path.cwd().parent
TUTORIAL_DIR = OPENEVOLVE_ROOT / "tutorial"

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_DIR = TUTORIAL_DIR / "runs" / f"tutorial_run_{timestamp}"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# CRITICAL: Ensure evals directory exists (needed by evaluator)
EVALS_DIR = TUTORIAL_DIR / "evals"
EVALS_DIR.mkdir(exist_ok=True)

print(f"✅ Output directory: {OUTPUT_DIR.relative_to(TUTORIAL_DIR)}")
print(f"✅ Evals directory: {EVALS_DIR.relative_to(TUTORIAL_DIR)}")

print("\n" + "="*70)
print("📋 Pre-Flight Check")
print("="*70)

try:
    kernel_var = INITIAL_KERNEL
    kernel_defined = True
except NameError:
    kernel_var = None
    kernel_defined = False

try:
    evaluator_var = EVALUATOR_PATH
    evaluator_defined = True
except NameError:
    evaluator_var = None
    evaluator_defined = False

try:
    config_var = CONFIG_FILE
    config_defined = True
except NameError:
    config_var = None
    config_defined = False

components = {
    "Kernel": (kernel_var, kernel_defined),
    "Evaluator": (evaluator_var, evaluator_defined),
    "Config": (config_var, config_defined)
}

all_ready = True
missing_cells = []

for name, (path, is_defined) in components.items():
    if not is_defined:
        print(f"❌ {name:12s}: NOT DEFINED (run earlier cell)")
        all_ready = False
        if name == "Kernel":
            missing_cells.append("Cell 5")
        elif name == "Evaluator":
            missing_cells.append("Cell 6")
        elif name == "Config":
            missing_cells.append("Cell 7")
    elif path and Path(path).exists():
        print(f"✅ {name:12s}: {Path(path).name}")
    else:
        print(f"❌ {name:12s}: NOT FOUND")
        all_ready = False

print("="*70)

if all_ready:
    print("\n🚀 All components ready! You can proceed to run evolution.")
else:
    print("\n⚠️  Some components are missing!")
    if missing_cells:
        print("\n📝 Please run these cells first:")
        for cell in missing_cells:
            print(f"   • {cell}")


✅ Output directory: runs/tutorial_run_20251126_143929
✅ Evals directory: evals

📋 Pre-Flight Check
✅ Kernel      : test_add_kernel.py
✅ Evaluator   : rocm_evaluator.py
✅ Config      : tutorial_config.yaml

🚀 All components ready! You can proceed to run evolution.


In [9]:
# Step 8: Run OpenEvolve Evolution
import subprocess
import os
from pathlib import Path

if not (INITIAL_KERNEL and EVALUATOR_PATH and CONFIG_FILE):
    print("❌ Missing required components!")
    print(f"   Kernel:    {INITIAL_KERNEL is not None and Path(INITIAL_KERNEL).exists()}")
    print(f"   Evaluator: {EVALUATOR_PATH is not None and Path(EVALUATOR_PATH).exists()}")
    print(f"   Config:    {CONFIG_FILE is not None and Path(CONFIG_FILE).exists()}")
else:
    command = [
        "openevolve-run",
        str(INITIAL_KERNEL),
        str(EVALUATOR_PATH),
        "--config", str(CONFIG_FILE),
        "--output", str(OUTPUT_DIR)
    ]
    
    print("🚀 Starting OpenEvolve Evolution...")
    print("="*70)
    print(f"📦 Kernel:    {Path(INITIAL_KERNEL).name}")
    print(f"⚙️  Evaluator: {Path(EVALUATOR_PATH).name}")
    print(f"📋 Config:    {Path(CONFIG_FILE).name}")
    print(f"📁 Output:    {OUTPUT_DIR.relative_to(TUTORIAL_DIR)}")
    print(f"🏠 Working Dir: {TUTORIAL_DIR}")
    print("="*70)
    print(f"\n$ cd {TUTORIAL_DIR}")
    print(f"$ {' '.join(command)}\n")
    print("="*70)
    
    # CRITICAL: Run from tutorial directory where evals/ exists
    result = subprocess.run(
        command, 
        capture_output=False, 
        text=True,
        cwd=str(TUTORIAL_DIR)  # Run from tutorial directory
    )
    
    print("="*70)
    if result.returncode == 0:
        print("\n✅ Evolution completed successfully!")
        print(f"\n📊 Results saved to: {OUTPUT_DIR.relative_to(TUTORIAL_DIR)}")
    else:
        print(f"\n❌ Evolution failed with exit code: {result.returncode}")


🚀 Starting OpenEvolve Evolution...
📦 Kernel:    test_add_kernel.py
⚙️  Evaluator: rocm_evaluator.py
📋 Config:    tutorial_config.yaml
📁 Output:    runs/tutorial_run_20251126_143929
🏠 Working Dir: /home/sapmajum/neurips/geak-openevolve/tutorial

$ cd /home/sapmajum/neurips/geak-openevolve/tutorial
$ openevolve-run /home/sapmajum/neurips/geak-openevolve/GEAK-eval-OE/geak_eval/data/ROCm/data/ROCm_v1/test_add_kernel.py /home/sapmajum/neurips/geak-openevolve/examples/tb/rocm_evaluator.py --config /home/sapmajum/neurips/geak-openevolve/tutorial/tutorial_config.yaml --output /home/sapmajum/neurips/geak-openevolve/tutorial/runs/tutorial_run_20251126_143929



2025-11-26 14:39:31,485 - INFO - Adding initial program to database
2025-11-26 14:39:37,581 - INFO - Time spent in evaluation: 6.09 seconds
2025-11-26 14:39:37,581 - INFO - Evaluated program 1847919d-3912-414e-a269-ebe918eb1bdb in 6.09s: success=1.0000, final_score=1.0000, performance_metrics=1.0000, correctness_score=1.0000, combined_score=1.0000, benchmark_results=['Performance report: Kernel parameters: SIZE=98432; BLOCK_SIZE_RUNTIME=1024; dtype_str=float16, achieved latency: 0.006900 ms, speedup: 1.0000x.'], baseline_comparison=Performance report: Kernel parameters: SIZE=98432; BLOCK_SIZE_RUNTIME=1024; dtype_str=float16, achieved latency: 0.006900 ms, speedup: 1.0000x. Speedup=1.0000x (baseline: 0.006900ms, current: 0.006900ms), individual_comparisons=[], summary=The Triton kernel was successfully compiled and launched. The Triton kernel produced correct results. The Triton kernel achieved a speedup of 1.0000x compared to the baseline., safety_validation=This generated Triton kerne

⚠️ WARNING: Kernel evaluator path /home/sapmajum/neurips/geak-openevolve/GEAK-eval-OE/geak_eval/data/ROCm/data/ROCm_v1/evaluator.py does not exist, using default given path.
🛡️ BULLETPROOF TRITON KERNEL EVALUATOR (AMD GPU) INITIALISED
Using program text from file: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpcpsuceea/test_add_kernel.py
Evaluating Triton kernel from: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpcpsuceea/test_add_kernel.py
Running correctness tests: pytest -v -x --maxfail=1 /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpcpsuceea/test_add_kernel.py -k not (test_performance or test_save)
Running performance tests: pytest -v -x --maxfail=1 /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpcpsuceea/test_add_kernel.py -k test_performance or test_save_performance_results
Performance test result - returncode: 0
Performance test stdout: ============================= test session starts ==============================
platform linux -- Pyth

2025-11-26 14:40:42,353 - INFO - Time spent in evaluation: 6.72 seconds
2025-11-26 14:40:42,353 - INFO - Evaluated program de87a355-acd9-45db-823b-9dcd13cf2339 in 6.72s: success=1.0000, final_score=0.4631, performance_metrics=0.4631, correctness_score=1.0000, combined_score=0.4631, benchmark_results=['Performance report: Kernel parameters: SIZE=98432; BLOCK_SIZE_RUNTIME=1024; dtype_str=float16, achieved latency: 0.014900 ms, speedup: 0.4631x.'], baseline_comparison=Performance report: Kernel parameters: SIZE=98432; BLOCK_SIZE_RUNTIME=1024; dtype_str=float16, achieved latency: 0.014900 ms, speedup: 0.4631x. Speedup=0.4631x (baseline: 0.006900ms, current: 0.014900ms), individual_comparisons=[], summary=The Triton kernel was successfully compiled and launched. The Triton kernel produced correct results. The Triton kernel achieved a speedup of 0.4631x compared to the baseline., safety_validation=This generated Triton kernel was evaluated safely, with no hard crashes or memory violations. T


Performance test stderr: 
Looking for performance results in: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpgwztd1rt/perf
Found 1 JSON files: ['add_kernel_perf.json']
Reading performance data from: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpgwztd1rt/perf/add_kernel_perf.json
Performance data structure: ['params', 'ms', 'min_ms', 'max_ms', 'GB/s', 'TFLOPS']
Performance: 0.0149ms (from key 'ms')
Loaded baseline latency from file: 0.006900ms
Calculated speedup: 0.006900ms / 0.014900ms = 0.4631x
🛡️ BULLETPROOF TRITON KERNEL EVALUATOR (AMD GPU) INITIALISED
Using program text from file: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmppt6h6pk1/test_add_kernel.py
Evaluating Triton kernel from: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmppt6h6pk1/test_add_kernel.py
Running correctness tests: pytest -v -x --maxfail=1 /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmppt6h6pk1/test_add_kernel.py -k not (test_performance or test_save)
Running p

...


Running correctness tests: pytest -v -x --maxfail=1 /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmph7r_9s8c/test_add_kernel.py -k not (test_performance or test_save)
Running performance tests: pytest -v -x --maxfail=1 /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpt_c42a9b/test_add_kernel.py -k test_performance or test_save_performance_results
Running performance tests: pytest -v -x --maxfail=1 /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmph7r_9s8c/test_add_kernel.py -k test_performance or test_save_performance_results
Performance test result - returncode: 0
Performance test stdout: ============================= test session starts ==============================
platform linux -- Python 3.13.9, pytest-9.0.1, pluggy-1.5.0 -- /home/sapmajum/miniconda3/bin/python3.13
cachedir: .pytest_cache
rootdir: /home/sapmajum/neurips/geak-openevolve
configfile: pyproject.toml
plugins: timeout-2.4.0, anyio-4.11.0
collecting ... collected 136 items / 7 deselected / 129 s

2025-11-26 14:43:31,707 - INFO - Time spent in evaluation: 3.19 seconds
2025-11-26 14:43:31,707 - INFO - Evaluated program 8c3f96a8-fff3-42aa-aa59-ed75fad6b4fb in 3.19s: success=0.0000, final_score=0.0000, error=Correctness tests failed (exit 1):
STDOUT: ============================= test session starts ==============================
platform linux -- Python 3.13.9, pytest-9.0.1, pluggy-1.5.0 -- /home/sapmajum/miniconda3/bin/python3.13
cachedir: .pytest_cache
rootdir: /home/sapmajum/neurips/geak-openevolve
configfile: pyproject.toml
plugins: timeout-2.4.0, anyio-4.11.0
collecting ... collected 136 items / 130 deselected / 6 selected

evals/tmpyddatjy0/test_add_kernel.py::test_add[64-128-64-128-float16] FAILED [ 16%]

=================================== FAILURES ===================================
_______________________ test_add[64-128-64-128-float16] ________________________

args = (constexpr[64], constexpr[8])
kwargs = {'_builder': <triton._C.libtriton.ir.builder object at 0x712c559


STDERR: 
🛡️ BULLETPROOF TRITON KERNEL EVALUATOR (AMD GPU) INITIALISED
Using program text from file: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpwumc3at2/test_add_kernel.py
Evaluating Triton kernel from: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpwumc3at2/test_add_kernel.py
Running correctness tests: pytest -v -x --maxfail=1 /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpwumc3at2/test_add_kernel.py -k not (test_performance or test_save)
Running performance tests: pytest -v -x --maxfail=1 /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmp1olrtcz1/test_add_kernel.py -k test_performance or test_save_performance_results
Running performance tests: pytest -v -x --maxfail=1 /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpwumc3at2/test_add_kernel.py -k test_performance or test_save_performance_results
Performance test result - returncode: 0
Performance test stdout: ============================= test session starts ===========================

2025-11-26 14:44:18,168 - INFO - Time spent in evaluation: 22.33 seconds
2025-11-26 14:44:18,169 - INFO - Evaluated program 9d28b1fd-7edf-4bbc-85ea-5fa4879088ee in 22.33s: success=1.0000, final_score=0.7582, performance_metrics=0.7582, correctness_score=1.0000, combined_score=0.7582, benchmark_results=['Performance report: Kernel parameters: SIZE=4096; M=64; N=64; BLOCK_SIZE_RUNTIME=4096; BLOCK_SIZE_M=64; BLOCK_SIZE_N=64; dtype_str=float16, achieved latency: 0.009100 ms, speedup: 0.7582x.'], baseline_comparison=Performance report: Kernel parameters: SIZE=4096; M=64; N=64; BLOCK_SIZE_RUNTIME=4096; BLOCK_SIZE_M=64; BLOCK_SIZE_N=64; dtype_str=float16, achieved latency: 0.009100 ms, speedup: 0.7582x. Speedup=0.7582x (baseline: 0.006900ms, current: 0.009100ms), individual_comparisons=[], summary=The Triton kernel was successfully compiled and launched. The Triton kernel produced correct results. The Triton kernel achieved a speedup of 0.7582x compared to the baseline., safety_validation=Thi


STDERR: 
Running performance tests: pytest -v -x --maxfail=1 /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpft66wgmd/test_add_kernel.py -k test_performance or test_save_performance_results
Running performance tests: pytest -v -x --maxfail=1 /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpqip8_or8/test_add_kernel.py -k test_performance or test_save_performance_results
Performance test result - returncode: 0
Performance test stdout: ============================= test session starts ==============================
platform linux -- Python 3.13.9, pytest-9.0.1, pluggy-1.5.0 -- /home/sapmajum/miniconda3/bin/python3.13
cachedir: .pytest_cache
rootdir: /home/sapmajum/neurips/geak-openevolve
configfile: pyproject.toml
plugins: timeout-2.4.0, anyio-4.11.0
collecting ... collected 136 items / 7 deselected / 129 selected

evals/tmpqip8_or8/test_add_kernel.py::test_performance[64-64-64-64-float16] PASSED 
Performance test stderr: 
Looking for performance results in: /home/sapmaj